# 標準化の実装

このノートブックでは、特徴量の標準化（Standardization）について学びます。

## 目次
1. [標準化の理論](#標準化の理論)
2. [基本的な実装](#基本的な実装)
3. [標準化の効果](#標準化の効果)
4. [実践的な応用例](#実践的な応用例)
5. [注意点とトラブルシューティング](#注意点とトラブルシューティング)
6. [演習問題](#演習問題)


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 乱数の固定
np.random.seed(42)

print("ライブラリのインポートが完了しました。")


## 標準化の理論

### 標準化とは
標準化（Standardization）は、データの平均を0、標準偏差を1に変換する手法です。

### 数学的表現
```
z = (x - μ) / σ
```

ここで：
- `x`: 元の値
- `μ`: 平均値
- `σ`: 標準偏差
- `z`: 標準化後の値

### 標準化の特徴
- **平均**: 0
- **標準偏差**: 1
- **分布**: 元の分布の形状を保持
- **外れ値**: 影響を受けやすい


## 基本的な実装

標準化の基本的な実装方法を学びます。


In [ ]:
# サンプルデータの生成
np.random.seed(42)

# 異なるスケールの特徴量を持つデータ
n_samples = 200
area = np.random.uniform(30, 150, n_samples)  # 面積 (m²)
age = np.random.uniform(0, 30, n_samples)      # 築年数 (年)
distance = np.random.uniform(0, 20, n_samples)  # 駅からの距離 (km)

# 価格の生成（線形関係 + ノイズ）
price = (50 * area + 100 * age - 10 * distance + 
         np.random.normal(0, 200, n_samples))

# データフレームの作成
data = pd.DataFrame({
    'area': area,
    'age': age,
    'distance': distance,
    'price': price
})

print("元データの基本統計:")
print(data.describe())

# データの可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 各特徴量の分布
features = ['area', 'age', 'distance', 'price']
titles = ['面積 (m²)', '築年数 (年)', '駅からの距離 (km)', '価格 (万円)']

for i, (feature, title) in enumerate(zip(features, titles)):
    row = i // 2
    col = i % 2
    axes[row, col].hist(data[feature], bins=20, alpha=0.7, edgecolor='black')
    axes[row, col].set_xlabel(title)
    axes[row, col].set_ylabel('頻度')
    axes[row, col].set_title(f'{title}の分布')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 特徴量のスケールの違いを確認
print(f"\n特徴量のスケール:")
for feature in ['area', 'age', 'distance']:
    mean_val = data[feature].mean()
    std_val = data[feature].std()
    print(f"{feature}: 平均={mean_val:.2f}, 標準偏差={std_val:.2f}")


In [ ]:
# 標準化の実装
# 特徴量と目的変数の分割
X = data[['area', 'age', 'distance']]
y = data['price']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"訓練データ: {X_train.shape}")
print(f"テストデータ: {X_test.shape}")

# 標準化の実行
scaler = StandardScaler()

# 訓練データでフィット（パラメータの学習）
X_train_scaled = scaler.fit_transform(X_train)

# テストデータに変換（学習したパラメータを使用）
X_test_scaled = scaler.transform(X_test)

# 標準化のパラメータを確認
print(f"\n標準化のパラメータ:")
print(f"平均: {scaler.mean_}")
print(f"標準偏差: {scaler.scale_}")

# 標準化前後の比較
print(f"\n標準化前の訓練データ統計:")
print(f"平均: {X_train.mean().values}")
print(f"標準偏差: {X_train.std().values}")

print(f"\n標準化後の訓練データ統計:")
print(f"平均: {X_train_scaled.mean(axis=0)}")
print(f"標準偏差: {X_train_scaled.std(axis=0)}")

# 標準化前後の可視化
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

features = ['area', 'age', 'distance']
titles = ['面積', '築年数', '駅からの距離']

for i, (feature, title) in enumerate(zip(features, titles)):
    # 標準化前
    axes[0, i].hist(X_train[feature], bins=20, alpha=0.7, color='blue', edgecolor='black')
    axes[0, i].set_xlabel(title)
    axes[0, i].set_ylabel('頻度')
    axes[0, i].set_title(f'標準化前: {title}')
    axes[0, i].grid(True, alpha=0.3)
    
    # 標準化後
    axes[1, i].hist(X_train_scaled[:, i], bins=20, alpha=0.7, color='red', edgecolor='black')
    axes[1, i].set_xlabel(f'{title} (標準化後)')
    axes[1, i].set_ylabel('頻度')
    axes[1, i].set_title(f'標準化後: {title}')
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 標準化の効果

標準化が機械学習アルゴリズムに与える影響を確認します。


In [ ]:
# 標準化の効果を比較
# 1. 標準化なしの線形回帰
lr_no_scale = LinearRegression()
lr_no_scale.fit(X_train, y_train)
y_pred_no_scale = lr_no_scale.predict(X_test)

# 2. 標準化ありの線形回帰
lr_scaled = LinearRegression()
lr_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = lr_scaled.predict(X_test_scaled)

# 性能の比較
mse_no_scale = mean_squared_error(y_test, y_pred_no_scale)
mse_scaled = mean_squared_error(y_test, y_pred_scaled)
r2_no_scale = r2_score(y_test, y_pred_no_scale)
r2_scaled = r2_score(y_test, y_pred_scaled)

print("標準化の効果:")
print("=" * 50)
print(f"標準化なし - MSE: {mse_no_scale:.2f}, R²: {r2_no_scale:.4f}")
print(f"標準化あり - MSE: {mse_scaled:.2f}, R²: {r2_scaled:.4f}")

# 回帰係数の比較
print(f"\n回帰係数の比較:")
print(f"標準化なし:")
print(f"  切片: {lr_no_scale.intercept_:.2f}")
for feature, coef in zip(['area', 'age', 'distance'], lr_no_scale.coef_):
    print(f"  {feature}: {coef:.2f}")

print(f"\n標準化あり:")
print(f"  切片: {lr_scaled.intercept_:.2f}")
for feature, coef in zip(['area', 'age', 'distance'], lr_scaled.coef_):
    print(f"  {feature}: {coef:.2f}")

# 係数の可視化
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 標準化なしの係数
axes[0].bar(['area', 'age', 'distance'], lr_no_scale.coef_, 
           color=['blue', 'green', 'red'], alpha=0.7)
axes[0].set_ylabel('回帰係数')
axes[0].set_title('標準化なしの回帰係数')
axes[0].grid(True, alpha=0.3)

# 標準化ありの係数
axes[1].bar(['area', 'age', 'distance'], lr_scaled.coef_, 
           color=['blue', 'green', 'red'], alpha=0.7)
axes[1].set_ylabel('回帰係数')
axes[1].set_title('標準化ありの回帰係数')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 予測値の比較
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 標準化なしの予測
axes[0].scatter(y_test, y_pred_no_scale, alpha=0.6, color='blue')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('実際の価格 (万円)')
axes[0].set_ylabel('予測価格 (万円)')
axes[0].set_title('標準化なしの予測')
axes[0].grid(True, alpha=0.3)

# 標準化ありの予測
axes[1].scatter(y_test, y_pred_scaled, alpha=0.6, color='red')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('実際の価格 (万円)')
axes[1].set_ylabel('予測価格 (万円)')
axes[1].set_title('標準化ありの予測')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 実践的な応用例

パイプラインを使用した実践的な標準化の実装例を示します。


In [ ]:
# パイプラインを使用した実装
# データリークを防ぐためのパイプライン
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# パイプラインの訓練
pipeline.fit(X_train, y_train)

# 予測
y_pred_pipeline = pipeline.predict(X_test)

# 性能の評価
mse_pipeline = mean_squared_error(y_test, y_pred_pipeline)
r2_pipeline = r2_score(y_test, y_pred_pipeline)

print("パイプラインの性能:")
print(f"MSE: {mse_pipeline:.2f}")
print(f"R²: {r2_pipeline:.4f}")

# パイプラインの詳細
print(f"\nパイプラインの詳細:")
print(f"ステップ1: {pipeline.named_steps['scaler']}")
print(f"ステップ2: {pipeline.named_steps['regressor']}")

# 新しいデータでの予測例
new_data = pd.DataFrame({
    'area': [100, 80, 120],
    'age': [10, 5, 15],
    'distance': [5, 10, 3]
})

print(f"\n新しいデータでの予測:")
print(f"入力データ:")
print(new_data)

# パイプラインを使用した予測
new_predictions = pipeline.predict(new_data)
print(f"\n予測結果:")
for i, pred in enumerate(new_predictions):
    print(f"サンプル {i+1}: {pred:.2f}万円")

# 交差検証での性能評価
from sklearn.model_selection import cross_val_score

# パイプラインを使用した交差検証
cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')

print(f"\n交差検証の結果:")
print(f"R²スコア: {cv_scores}")
print(f"平均R²: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# 異なるスケーリング手法の比較
def compare_scalers():
    """異なるスケーリング手法の比較"""
    scalers = {
        'StandardScaler': StandardScaler(),
        'MinMaxScaler': MinMaxScaler(),
        'RobustScaler': RobustScaler()
    }
    
    results = {}
    
    for name, scaler in scalers.items():
        # パイプラインの作成
        pipe = Pipeline([
            ('scaler', scaler),
            ('regressor', LinearRegression())
        ])
        
        # 交差検証
        scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')
        results[name] = scores.mean()
    
    return results

# スケーリング手法の比較
scaler_results = compare_scalers()

print(f"\nスケーリング手法の比較:")
for name, score in scaler_results.items():
    print(f"{name}: R² = {score:.4f}")

# 可視化
plt.figure(figsize=(10, 6))
names = list(scaler_results.keys())
scores = list(scaler_results.values())

bars = plt.bar(names, scores, color=['blue', 'green', 'red'], alpha=0.7)
plt.ylabel('R²スコア')
plt.title('スケーリング手法の性能比較')
plt.grid(True, alpha=0.3)

# スコアをバーに表示
for bar, score in zip(bars, scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
             f'{score:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


## 注意点とトラブルシューティング

標準化を実装する際の注意点とよくある問題について説明します。


In [ ]:
# 注意点1: データリークの例
print("=== 注意点1: データリークの例 ===")

# 間違った方法（データリーク）
print("間違った方法:")
X_all = data[['area', 'age', 'distance']]
y_all = data['price']

# 全データでスケーリング（間違い）
scaler_wrong = StandardScaler()
X_all_scaled = scaler_wrong.fit_transform(X_all)

# 訓練・テスト分割（間違い）
X_train_wrong, X_test_wrong, y_train_wrong, y_test_wrong = train_test_split(
    X_all_scaled, y_all, test_size=0.2, random_state=42
)

print(f"全データでスケーリング後の統計:")
print(f"訓練データ平均: {X_train_wrong.mean(axis=0)}")
print(f"テストデータ平均: {X_test_wrong.mean(axis=0)}")

# 正しい方法
print("\n正しい方法:")
X_train_correct, X_test_correct, y_train_correct, y_test_correct = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 訓練データのみでスケーリング
scaler_correct = StandardScaler()
X_train_correct_scaled = scaler_correct.fit_transform(X_train_correct)
X_test_correct_scaled = scaler_correct.transform(X_test_correct)

print(f"訓練データのみでスケーリング後の統計:")
print(f"訓練データ平均: {X_train_correct_scaled.mean(axis=0)}")
print(f"テストデータ平均: {X_test_correct_scaled.mean(axis=0)}")

# 注意点2: 外れ値の影響
print("\n=== 注意点2: 外れ値の影響 ===")

# 外れ値を含むデータの生成
np.random.seed(42)
X_outlier = X.copy()
X_outlier.iloc[0, 0] = 1000  # 面積に外れ値を追加

print("外れ値ありデータの統計:")
print(X_outlier.describe())

# 標準化の実行
scaler_outlier = StandardScaler()
X_outlier_scaled = scaler_outlier.fit_transform(X_outlier)

print(f"\n外れ値ありデータの標準化後統計:")
print(f"平均: {X_outlier_scaled.mean(axis=0)}")
print(f"標準偏差: {X_outlier_scaled.std(axis=0)}")

# 外れ値の影響を可視化
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 元データ
axes[0].scatter(X['area'], X['age'], alpha=0.6, color='blue', label='正常データ')
axes[0].scatter(X_outlier.iloc[0, 0], X_outlier.iloc[0, 1], color='red', s=100, label='外れ値')
axes[0].set_xlabel('面積')
axes[0].set_ylabel('築年数')
axes[0].set_title('元データ（外れ値あり）')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 標準化後
X_normal_scaled = scaler_correct.fit_transform(X)
axes[1].scatter(X_normal_scaled[:, 0], X_normal_scaled[:, 1], alpha=0.6, color='blue', label='正常データ')
axes[1].scatter(X_outlier_scaled[0, 0], X_outlier_scaled[0, 1], color='red', s=100, label='外れ値')
axes[1].set_xlabel('面積（標準化後）')
axes[1].set_ylabel('築年数（標準化後）')
axes[1].set_title('標準化後（外れ値あり）')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 注意点3: 新しいデータの処理
print("\n=== 注意点3: 新しいデータの処理 ===")

# 新しいデータ（範囲外の値）
new_data_outlier = pd.DataFrame({
    'area': [200, 50, 300],  # 範囲外の値
    'age': [35, 5, 40],      # 範囲外の値
    'distance': [25, 2, 30]  # 範囲外の値
})

print("新しいデータ（範囲外の値）:")
print(new_data_outlier)

# 標準化の適用
new_data_scaled = scaler_correct.transform(new_data_outlier)

print(f"\n標準化後の新しいデータ:")
print(new_data_scaled)

# 注意点4: カテゴリカル変数の処理
print("\n=== 注意点4: カテゴリカル変数の処理 ===")

# カテゴリカル変数を含むデータ
categorical_data = pd.DataFrame({
    'area': [80, 100, 120, 90, 110],
    'age': [5, 10, 15, 8, 12],
    'district': ['A', 'B', 'A', 'C', 'B'],  # カテゴリカル変数
    'price': [4000, 5000, 6000, 4500, 5500]
})

print("カテゴリカル変数を含むデータ:")
print(categorical_data)

# カテゴリカル変数のエンコーディング
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
categorical_data['district_encoded'] = le.fit_transform(categorical_data['district'])

print(f"\nエンコーディング後:")
print(categorical_data)

# 数値変数のみを標準化
X_cat = categorical_data[['area', 'age', 'district_encoded']]
y_cat = categorical_data['price']

scaler_cat = StandardScaler()
X_cat_scaled = scaler_cat.fit_transform(X_cat)

print(f"\n標準化後:")
print(f"平均: {X_cat_scaled.mean(axis=0)}")
print(f"標準偏差: {X_cat_scaled.std(axis=0)}")

# 注意点5: 欠損値の処理
print("\n=== 注意点5: 欠損値の処理 ===")

# 欠損値を含むデータ
missing_data = data.copy()
missing_data.iloc[0, 0] = np.nan  # 面積に欠損値を追加
missing_data.iloc[1, 1] = np.nan  # 築年数に欠損値を追加

print("欠損値ありデータ:")
print(missing_data.head())

# 欠損値の処理
from sklearn.impute import SimpleImputer

# 欠損値の補完
imputer = SimpleImputer(strategy='mean')
X_missing = missing_data[['area', 'age', 'distance']]
X_missing_imputed = imputer.fit_transform(X_missing)

print(f"\n欠損値補完後:")
print(f"欠損値の数: {np.isnan(X_missing_imputed).sum()}")

# 標準化の実行
scaler_missing = StandardScaler()
X_missing_scaled = scaler_missing.fit_transform(X_missing_imputed)

print(f"\n標準化後:")
print(f"平均: {X_missing_scaled.mean(axis=0)}")
print(f"標準偏差: {X_missing_scaled.std(axis=0)}")


## 演習問題

### 問題1: 外れ値の影響
外れ値を含むデータで標準化を実行し、RobustScalerとの性能を比較してください。

### 問題2: カテゴリカル変数の処理
カテゴリカル変数を含むデータセットで、適切な前処理と標準化を実装してください。

### 問題3: パイプラインの最適化
異なるスケーリング手法とアルゴリズムの組み合わせで、最適なパイプラインを見つけてください。

### 問題4: 実データでの検証
実際のデータセット（Boston Housing、California Housing等）で標準化の効果を検証してください。
